# 1. Introducción y Preparación del Entorno
En esta aproximación al problema de **Summarization**, utilizaremos técnicas de **Shallow Learning**.

Vamos a utilizar técnicas de **Sparse Representations**. A diferencia de los modelos de Deep Learning que generan texto nuevo palabra por palabra, aquí nos centraremos en **Extractive Summarization**: seleccionar las frases más relevantes del diálogo original y concatenarlas para formar un resumen.

**Decisiones de Diseño:**
* **Dataset:** Utilizamos el set de entrenamiento de "MTS-Dialog".
* **Preprocesamiento:** Para esta tarea, la unidad mínima de información no es el token, sino la sentence. Usamos "sent_tokenize" de NLTK en lugar de dividir por puntos simples para manejar mejor las abreviaturas médicas (por ejemplo, "Dr." o "Mr.") y evitar cortes erróneos.

In [25]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

# Descargar recursos necesarios de NLTK
nltk.download('punkt')
!pip install rouge-score

# Cargar datos
df_train = pd.read_csv("../../dataset/MTS-Dialog-TrainingSet.csv")
df_sample = df_train.head(100).copy() 

print("Datos cargados. Ejemplo de diálogo:")
print(df_sample['dialogue'].iloc[0][:200] + "...")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Iker\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Defaulting to user installation because normal site-packages is not writeable
Datos cargados. Ejemplo de diálogo:
Doctor: What brings you back into the clinic today, miss? 
Patient: I came in for a refill of my blood pressure medicine. 
Doctor: It looks like Doctor Kumar followed up with you last time regarding...


# 2. Técnica 1: Resumen basado en Frecuencia (TF-IDF)
Nuestra primera técnica utiliza **TF-IDF** como medida de importancia.

**Hipótesis:**
En un diálogo médico, las palabras más informativas son aquellas que son específicas del caso ("diabetes", "ibuprofeno", "cirugía") y no las palabras comunes de relleno ("hola", "bueno", "entonces").

**Metodología:**
1.  Convertimos el diálogo en una matriz TF-IDF dispersa.
2.  Calculamos el score de cada frase sumando los valores TF-IDF de las palabras que contiene.
3.  Seleccionamos las N frases con mayor puntuación acumulada.

TF-IDF genera vectores de alta dimensionalidad con muchos ceros, ignorando el orden semántico profundo pero capturando palabras clave críticas.

In [26]:
def summarize_tfidf(text, num_sentences=2):
    # 1. Dividir en frases
    sentences = sent_tokenize(text)
    if len(sentences) <= num_sentences:
        return text
    
    # 2. Calcular TF-IDF
    vectorizer = TfidfVectorizer(stop_words='english')
    try:
        tfidf_matrix = vectorizer.fit_transform(sentences)
    except ValueError: # Caso textos vacíos
        return text
        
    # 3. Sumar scores por frase
    sentence_scores = np.sum(tfidf_matrix.toarray(), axis=1)
    
    # 4. Ordenar y elegir las top N frases
    top_indices = np.argsort(sentence_scores)[-num_sentences:]
    top_indices = sorted(top_indices) # Reordenar para mantener flujo original
    
    summary = " ".join([sentences[i] for i in top_indices])
    return summary

# Aplicar al dataset
df_train['pred_tfidf'] = df_train['dialogue'].apply(lambda x: summarize_tfidf(str(x)))
print("Resumen TF-IDF generado para el primer ejemplo:")
print(df_train['pred_tfidf'].iloc[0])

Resumen TF-IDF generado para el primer ejemplo:
Doctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones. Doctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure?


# 3. Técnica 2: Resumen basado en Grafos (TextRank)
Como segunda técnica de Shallow Learning, implementamos **TextRank**, un algoritmo no supervisado inspirado en PageRank.

**Justificación y Diferencias:**
A diferencia de TF-IDF, que mira palabras aisladas, TextRank considera la "relación entre frases".
* Modelamos el texto como un grafo donde los nodos son las oraciones.
* Las conexiones representan la cosine-similarity entre los vectores TF-IDF de las oraciones.

**Hipótesis:**
Las oraciones más importantes no son necesariamente las que tienen palabras raras (como en TF-IDF), sino las que son más representativas del contenido global del diálogo (aquellas que se parecen más al resto de las oraciones). Esto debería producir resúmenes más cohesivos.

In [27]:
def summarize_textrank(text, num_sentences=2):
    sentences = sent_tokenize(text)
    if len(sentences) <= num_sentences:
        return text

    # 1. Crear matriz de similaridad entre frases (usando TF-IDF features)
    vectorizer = TfidfVectorizer(stop_words='english')
    try:
        tfidf_matrix = vectorizer.fit_transform(sentences)
    except ValueError:
        return text
    
    similarity_matrix = cosine_similarity(tfidf_matrix)
    
    # 2. Crear grafo y calcular PageRank
    nx_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(nx_graph)
    
    # 3. Ordenar por puntuación
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    
    # 4. Seleccionar top frases
    summary = " ".join([s for score, s in ranked_sentences[:num_sentences]])
    return summary

# Aplicar al dataset
df_train['pred_textrank'] = df_train['dialogue'].apply(lambda x: summarize_textrank(str(x)))
print("Resumen TextRank generado.")

Resumen TextRank generado.


# 4. Evaluación Comparativa (Métricas ROUGE)
Para evaluar la calidad de los resúmenes, no vamos a usar "Accuracy". Utilizamos ROUGE, que es el estándar en NLP para esta tarea.

* **ROUGE-1:** Mide la coincidencia de unigramas (palabras individuales). Evalúa la cobertura de contenido, es decir, si se mencionan las palabras clave.
* **ROUGE-L:** Mide la subsecuencia común más larga. Evalúa la fluidez y estructura a nivel de frase.

**Nota sobre los Resultados Esperados:**
Esperamos puntuaciones bajas (entorno a 0.15 - 0.25) debido a la naturaleza del problema:
1.  **Gap Extractivo vs Abstractivo:** Estamos intentando reconstruir una nota clínica formal recortando frases de un diálogo informal.
2.  **Penalización:** ROUGE penaliza severamente si no usamos las palabras exactas. Si el paciente dice "me duele la tripa" y el médico anota "dolor abdominal", nuestros modelos de Shallow Learning fallarán en capturar esa relación, bajando el score.

In [28]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

def calculate_scores(preds, refs):
    scores = {'rouge1': [], 'rougeL': []}
    for p, r in zip(preds, refs):
        score = scorer.score(str(r), str(p))
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)
    return {k: np.mean(v) for k, v in scores.items()}

# Evaluar TF-IDF
scores_tfidf = calculate_scores(df_train['pred_tfidf'], df_train['section_text'])
print("--- Resultados Shallow ML 1: TF-IDF ---")
print(scores_tfidf)

# Evaluar TextRank
scores_textrank = calculate_scores(df_train['pred_textrank'], df_train['section_text'])
print("\n--- Resultados Shallow ML 2: TextRank ---")
print(scores_textrank)

--- Resultados Shallow ML 1: TF-IDF ---
{'rouge1': np.float64(0.20246636087274483), 'rougeL': np.float64(0.16167682145568624)}

--- Resultados Shallow ML 2: TextRank ---
{'rouge1': np.float64(0.10599440719873192), 'rougeL': np.float64(0.08904900648066023)}


## Análisis de Resultados (Shallow Learning)

Se puede ver una clara superioridad de TF-IDF sobre TextRank.

**Justificación Técnica:**
1.  **Alineación Semántica:** TF-IDF premia términos específicos que tienen alta carga informativa y suelen aparecer tal cual en el resumen médico.
2.  **Fallo de TextRank:** Al basarse en grafos de similitud, TextRank creemos que confunde frases estructurales o de "relleno" (saludos, confirmaciones) como las más importantes por su centralidad en el diálogo, introduciendo ruido.
3.  **Limitación Extractiva:** Ambos modelos tienen un techo de rendimiento bajo porque no pueden parafrasear. Como hemos mencionado anteriormente, si el paciente dice "dolor de tripa" y el médico escribe "dolor abdominal", estos modelos fallan al buscar coincidencia exacta, lo que justifica cambiar a metodos de DL mas avanzados.